In [1]:
import import_ipynb

In [14]:
# code stub
json_dict = """{
  "nodes": {
    "__START__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The starting point of the graph.",
      "function_name": "start_node"
    },
    "supervisor": {
      "schema_info": "Router: TypedDict with next as a string literal of options.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node manages the conversation between workers and decides which worker to route to next.",
      "function_name": "supervisor_node"
    },
    "food_logger": {
      "schema_info": "State: TypedDict with messages as a list of messages.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs food intake and calculates calories.",
      "function_name": "food_logger_node"
    },
    "workout_tracker": {
      "schema_info": "State: TypedDict with messages as a list of messages.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs workout details and calculates calories burnt.",
      "function_name": "workout_tracker_node"
    },
    "__END__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The endpoint of the graph, indicating completion.",
      "function_name": "end_node"
    }
  },
  "edges": {
    { 
      "source": "__START__", 
      "target": "supervisor", 
      "routing_conditions": "Start the process by routing to the supervisor node.", 
      "conditional": False 
    },
    { 
      "source": "supervisor", 
      "target": "food_logger", 
      "routing_conditions": "If the supervisor decides to route to food_logger based on user input.", 
      "conditional": True 
    },
    { 
      "source": "supervisor", 
      "target": "workout_tracker", 
      "routing_conditions": "If the supervisor decides to route to workout_tracker based on user input.", 
      "conditional": True 
    },
    { 
      "source": "supervisor", 
      "target": "__END__", 
      "routing_conditions": "If the supervisor decides to finish the process.", 
      "conditional": True
    },
    { 
      "source": "food_logger", 
      "target": "supervisor", 
      "routing_conditions": "After logging food intake, return control to the supervisor.", 
      "conditional": False 
    },
    { 
      "source": "workout_tracker", 
      "target": "supervisor", 
      "routing_conditions": "After logging workout details, return control to the supervisor.", 
      "conditional": False 
    }
  }
}
"""

code = """ 
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
model = ChatOpenAI(model="gpt-4o-mini")

# Create specialized agents

@tool
def food_logger_tool(food: str) -> str:
    \"\"\"Log food intake in the database\"\"\"
    # Parse food input and get nutrition data from Nutritionix API
    import requests
    import json
    from pymongo import MongoClient
    from datetime import datetime

    # Nutritionix API credentials should be in environment variables
    headers = {
        "x-app-id": os.getenv("NUTRITIONIX_APP_ID"),
        "x-app-key": os.getenv("NUTRITIONIX_API_KEY"),
        "x-remote-user-id": "0"  # 0 for development
    }

    # Query Nutritionix API
    endpoint = "https://trackapi.nutritionix.com/v2/natural/nutrients"
    payload = {"query": food}
    
    try:
        response = requests.post(endpoint, headers=headers, json=payload)
        nutrition_data = response.json()

        # Connect to MongoDB
        client = MongoClient(os.getenv("MONGODB_URI"))
        db = client.nutrition_db
        food_logs = db.food_logs

        # Prepare document for MongoDB
        food_entry = {
            "food_name": food,
            "timestamp": datetime.now(),
            "nutrition_data": nutrition_data["foods"][0],
            "calories": nutrition_data["foods"][0]["nf_calories"],
            "protein": nutrition_data["foods"][0]["nf_protein"],
            "carbohydrates": nutrition_data["foods"][0]["nf_total_carbohydrate"],
            "fat": nutrition_data["foods"][0]["nf_total_fat"]
        }

        # Insert into MongoDB
        food_logs.insert_one(food_entry)
        client.close()

    except Exception as e:
        return f"Error logging food: {str(e)}"

@tool
def workout_tracker_tool(workout: str) -> str:
    \"\"\"Log workout details in the database\"\"\"
    # Parse workout input and get workout details from Google Fit API
    import requests
    import json
    from pymongo import MongoClient
    from datetime import datetime

    # Google Fit API credentials should be in environment variables
    headers = {
        "Authorization": f"Bearer {os.getenv('GOOGLE_FIT_TOKEN')}"
    }

    # Query Google Fit API
    endpoint = "https://www.googleapis.com/fitness/v1/users/me/dataset:aggregate"
    
    try:
        # Connect to MongoDB
        client = MongoClient(os.getenv("MONGODB_URI"))
        db = client.fitness_db
        workout_logs = db.workout_logs

        # Prepare document for MongoDB
        workout_entry = {
            "workout_type": workout,
            "timestamp": datetime.now(),
            "duration_minutes": 30,  # This would come from Google Fit API
            "calories_burned": 250,  # This would come from Google Fit API
            "heart_rate_avg": 140,   # This would come from Google Fit API
            "steps": 4000            # This would come from Google Fit API
        }

        # Insert into MongoDB
        workout_logs.insert_one(workout_entry)
        client.close()
        
        return f"Successfully logged workout: {workout}"

    except Exception as e:
        return f"Error logging workout: {str(e)}"

food_logger_agent = create_react_agent(
    model=model,
    tools=[food_logger_tool],
    name="food_logger",
    prompt="user will tell you what they ate and you will log it in the database"
)

workout_tracker_agent = create_react_agent(
    model=model,
    tools=[workout_tracker_tool],
    name="workout_tracker",
    prompt="user will tell you what they did and you will log it in the database"
)

# Create supervisor workflow
workflow = create_supervisor(
    [food_logger_agent, workout_tracker_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For tracking calories, use food_logger "
        "For tracking workout, use workout_tracker"
    )
)

# Compile and run
app = workflow.compile()
"""

In [2]:
# generate use cases
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
SYS_PROMPT= ChatPromptTemplate.from_template("""
You are given the json of a workflow graph below.
{json_dict}
You are supposed to write use cases for the graph.
You will also do dry run of the graph with the use cases.
The use cases should be in the format of a list of dictionaries.
Each dictionary should have the following
keys:
- name: The name of the use case
- description: The description of the use case
- dry_run: The dry run of the use case
""")

from model_factory import get_model
llm = get_model()
use_cases = llm.invoke([SystemMessage(content=SYS_PROMPT.format(json_dict=json_dict))])
use_cases.pretty_print()

================================== Ai Message ==================================

[
  {
    "name": "Log Food Intake and Finish",
    "description": "User logs their food intake and then finishes the process.",
    "dry_run": [
      {"node": "__START__", "action": "Start the process by routing to the supervisor node."},
      {"node": "supervisor", "action": "User chooses to log food intake, route to food_logger."},
      {"node": "food_logger", "action": "Log food intake and calculate calories."},
      {"node": "food_logger", "action": "Return control to supervisor."},
      {"node": "supervisor", "action": "User decides to finish the process, route to __END__."},
      {"node": "__END__", "action": "End the process."}
    ]
  },
  {
    "name": "Log Workout and Finish",
    "description": "User logs their workout details and then finishes the process.",
    "dry_run": [
      {"node": "__START__", "action": "Start the process by routing to the supervisor node."},
      {"node": "su

In [ ]:
# mock generation
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate

SYS_PROMPT = """
You are given a code, look for any methods with 'tool' decorator and modify them to have mock implementation.

Return the modified code in the same format as the input code.
"""


mocked_code = llm.invoke([SystemMessage(content=SYS_PROMPT), HumanMessage(content=code)])
mocked_code.pretty_print()

================================== Ai Message ==================================

 
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
model = ChatOpenAI(model="gpt-4o-mini")

# Create specialized agents

@tool
def food_logger_tool(food: str) -> str:
    """Mock implementation of food_logger_tool"""
    return f"Mock log: Food '{food}' has been logged successfully."

@tool
def workout_tracker_tool(workout: str) -> str:
    """Mock implementation of workout_tracker_tool"""
    return f"Mock log: Workout '{workout}' has been logged successfully."

food_logger_agent = create_react_agent(
    model=model,
    tools=[food_logger_tool],
    name="food_logger",
    prompt="user will tell you what they ate and you will log it in the database"
)

workout_tracker_agent = create_react_agent(
    model=model,
    tools=[workout_tracker_tool],
    name="wor

In [16]:
from bs4 import BeautifulSoup
import html2text
import httpx

def fetch_documents(url: str) -> str:
    """Fetch a document from a URL and return the markdownified text.

    Args:
        url (str): The URL of the document to fetch.

    Returns:
        str: The markdownified text of the document.
    """
    httpx_client = httpx.Client(follow_redirects=True, timeout=10)

    try:
        response = httpx_client.get(url, timeout=10)
        response.raise_for_status()
        html_content = response
        soup = BeautifulSoup(html_content, 'html.parser')
    
        img_tags = soup.find_all('img')
        for img_tag in img_tags:
            img_tag.decompose()

        target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain
        
        if not target_div:
            target_div = soup.find('article') #langraph

        if not target_div:
            target_div = soup.find('html') #langraph

        if not target_div:
            return html2text.html2text(str(soup))
        
        return html2text.html2text(str(target_div))
    except (httpx.HTTPStatusError, httpx.RequestError) as e:
        return f"Encountered an HTTP error: {str(e)}"

In [25]:
from model_factory import ModelName
# generate test cases
TEST_GEN_PROMPT = ChatPromptTemplate.from_template("""
You are given langgraph code below:
<CODE>
{code}
</CODE>
You are given the use cases for a workflow graph along with dry runs.
<USE_CASES>
{use_cases}
</USE_CASES>
                                                   
Below are documents which contain information about how to write test cases for the graph.
<DOCUMENTS>
<DOCUMENT1>
{document1}
</DOCUMENT1>
</DOCUMENTS>
                                                   
You are supposed to write test cases for the graph in the <CODE> section, use the <USE_CASES> for generating test case inputs.
                                              
You are supposed to see the kind of tests that are being written in the <DOCUMENTS> section and write your own test cases in the same format.
The tests should cover the following:
1. Final response: Use LLM-as-judge to grade the final response of the graph.
2. Trajectory: The trajectory of the nodes and tools called within the graph.
3.Single step: Depending on the nature of each node, either using LLM-as-judge if the output of Node is a LLM text response, if the LLM is binded with tools, check if the llm is calling the correct tool when required and just responding back when not required.

For the 3 type of tests the goal is to use pytest.parameterize to test against the different kind of inputs that could be derived from use cases.                  
Note:
- Also include the code in the output at the top
                                                   
DONT WRITE ANYTHING ELSE IN THE OUTPUT, ONLY OUTPUT THE CODE
THIS CODE IS THE CODE THAT WILL BE RUN, SO IT SHOULD BE VALID PYTHON CODE
DONT WRITE TOO MANY COMMENTS IN THE CODE, KEEP IT SIMPLE
""")


gemini_llm = get_model(ModelName.GEMINI25FLASH)
final_file =gemini_llm.invoke(TEST_GEN_PROMPT.format(
    code=mocked_code,
    use_cases=use_cases,
    document1=fetch_documents("https://docs.smith.langchain.com/evaluation/tutorials/testing")
    ))

final_file.pretty_print()

================================== Ai Message ==================================

```python
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
import pytest
from langsmith import testing as t
from typing_extensions import Annotated, TypedDict
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemoryCheckpoint
import asyncio

# --- Original Code Block ---
# Assume this code is in a file named app.py or similar
# For testing purposes, we'll include it directly or ensure it's importable.

model = ChatOpenAI(model="gpt-4o-mini")

@tool
def food_logger_tool(food: str) -> str:
    """Mock implementation of food_logger_tool"""
    return f"Mock log: Food '{food}' has been logged successfully."

@tool
def workout_tracker_tool(workout: str) -> str:
    """Mock implementation of

In [ ]:
# run in e2b pipeline